In [1]:
import numpy as np
import pandas as pd
import io
import csv
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

# 1. Загрузим данные


Загрузим таги и сразу сделаем копию

In [2]:
tags = pd.read_csv('/content/drive/MyDrive/dataset/Tags.csv')

In [3]:
tags_copy = tags.copy

In [4]:
tags.head()

,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn


Ну оно ж зависит друг от друга


In [5]:
tags = (tags.groupby('Id')['Tag'].apply(list).reset_index())

In [6]:
tags.head()

,Id,Tag
0,80,"[flex, actionscript-3, air]"
1,90,"[svn, tortoisesvn, branch, branching-and-merging]"
2,120,"[sql, asp.net, sitemap]"
3,180,"[algorithm, language-agnostic, colors, color-s..."
4,260,"[c#, .net, scripting, compiler-construction]"


Загрузим ответы

In [7]:
ans = pd.read_csv('/content/drive/MyDrive/dataset/Answers.csv', sep=',', quotechar='"', engine='python', encoding='latin-1', skipinitialspace=True, on_bad_lines='skip')

In [8]:
ans.head()

,Id,OwnerUserId,CreationDate,ParentId,Score,Body
0,92,61.0,2008-08-01T14:45:37Z,90,13,"<p><a href=""http://svnbook.red-bean.com/"">Vers..."
1,124,26.0,2008-08-01T16:09:47Z,80,12,<p>I wound up using this. It is a kind of a ha...
2,199,50.0,2008-08-01T19:36:46Z,180,1,<p>I've read somewhere the human eye can't dis...
3,269,91.0,2008-08-01T23:49:57Z,260,4,"<p>Yes, I thought about that, but I soon figur..."
4,307,49.0,2008-08-02T01:49:46Z,260,28,"<p><a href=""http://www.codeproject.com/Article..."


Загрузим вопросы

In [9]:
ques = pd.read_csv('/content/drive/MyDrive/dataset/Questions.csv', sep=',', quotechar='"', engine='python', encoding='latin-1', skipinitialspace=True, on_bad_lines='skip')

In [10]:
ques.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


Cмерджим данные по айдишникам



In [11]:
questags = pd.merge(ques, tags, on='Id', how='left')

In [12]:
questags.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body,Tag
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,"[flex, actionscript-3, air]"
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,"[svn, tortoisesvn, branch, branching-and-merging]"
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,"[sql, asp.net, sitemap]"
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,"[algorithm, language-agnostic, colors, color-s..."
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,"[c#, .net, scripting, compiler-construction]"


In [13]:
df = pd.merge(questags, ans, left_on='Id', right_on='ParentId', how='left', suffixes=('_questions', '_answers'))

Теперь красота, ток удалим лишние данные


In [14]:
df.head()

,Id_questions,OwnerUserId_questions,CreationDate_questions,ClosedDate,Score_questions,Title,Body_questions,Tag,Id_answers,OwnerUserId_answers,CreationDate_answers,ParentId,Score_answers,Body_answers
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,"[flex, actionscript-3, air]",124.0,26.0,2008-08-01T16:09:47Z,80.0,12.0,<p>I wound up using this. It is a kind of a ha...
1,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,"[flex, actionscript-3, air]",10008.0,1109.0,2008-08-13T16:09:09Z,80.0,6.0,"<p>The <a href=""http://en.wikipedia.org/wiki/S..."
2,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,"[flex, actionscript-3, air]",3770976.0,364174.0,2010-09-22T15:37:30Z,80.0,1.0,<p>What about making your delimiter something ...
3,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,"[svn, tortoisesvn, branch, branching-and-merging]",92.0,61.0,2008-08-01T14:45:37Z,90.0,13.0,"<p><a href=""http://svnbook.red-bean.com/"">Vers..."
4,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,"[svn, tortoisesvn, branch, branching-and-merging]",202317.0,20709.0,2008-10-14T18:41:45Z,90.0,2.0,"<p>You can also try <em><a href=""http://www.co..."


In [15]:
df_full = df.drop(columns=['ParentId'])
df_full = df.drop(columns=['CreationDate_questions'])
df_full = df.drop(columns=['ClosedDate'])
df_full = df.drop(columns=['CreationDate_answers'])

In [16]:
df_full.head()

,Id_questions,OwnerUserId_questions,CreationDate_questions,ClosedDate,Score_questions,Title,Body_questions,Tag,Id_answers,OwnerUserId_answers,ParentId,Score_answers,Body_answers
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,"[flex, actionscript-3, air]",124.0,26.0,80.0,12.0,<p>I wound up using this. It is a kind of a ha...
1,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,"[flex, actionscript-3, air]",10008.0,1109.0,80.0,6.0,"<p>The <a href=""http://en.wikipedia.org/wiki/S..."
2,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,"[flex, actionscript-3, air]",3770976.0,364174.0,80.0,1.0,<p>What about making your delimiter something ...
3,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,"[svn, tortoisesvn, branch, branching-and-merging]",92.0,61.0,90.0,13.0,"<p><a href=""http://svnbook.red-bean.com/"">Vers..."
4,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,"[svn, tortoisesvn, branch, branching-and-merging]",202317.0,20709.0,90.0,2.0,"<p>You can also try <em><a href=""http://www.co..."


# 2. Очистим текст

Очистим с помощью регулярок и обработки nlp

In [17]:
import re

In [18]:
import spacy

In [19]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

С помощью регулярок убираем HTML, Markdown-ссылки и код

In [20]:
def clean_html_md(text: str) -> str:
    text = text or ''
    soup = BeautifulSoup(text, 'html.parser')
    text = soup.get_text(separator=' ')
    text = re.sub(r'\[([^\]]+)\]\([^)]+\)', r'\1', text)
    text = re.sub(r'`{1,3}.*?`{1,3}', ' ', text, flags=re.DOTALL)
    text = re.sub(r'<[^>]+>', ' ', text)
    return text

Нормализуем, оставив пробелы и символы

In [21]:
def normalize(text: str) -> str:
    text = text.lower()
    text = re.sub(r'[^a-z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

Ну лемматизация и лемматизация че бубнить то

In [22]:
def lemmatize(text: str) -> str:
    doc = nlp(text)
    lemmas = [
        tok.lemma_
        for tok in doc
        if tok.is_alpha and not tok.is_stop
    ]
    return ' '.join(lemmas)

In [23]:
def preprocess(text: str) -> str:
    text = clean_html_md(text)
    text = normalize(text)
    text = lemmatize(text)
    return text

Как же оно долго работает, вот бы в дедлайн успеть

In [ ]:
df_full['clean_question'] = (df_full['Body_questions'].fillna('').apply(preprocess))
df_full['clean_answer'] = (df_full['Body_answers'].fillna('').apply(preprocess))

Сохраним очищенный датасет для дальнейшей работы и дальнейших лаб


In [ ]:
df_full.to_csv('/content/drive/MyDrive/dataset/df_full.csv', index=False)

Считаем его


In [ ]:
df_full = pd.read_csv('/content/drive/MyDrive/dataset/df_full.csv')

Выводим


In [ ]:
df_full.head()

Оставим только очищенные данные

In [ ]:
df_full = df_full.drop(columns=['Body_questions'])

In [ ]:
df_full = df_full.drop(columns=['Body_answers'])

In [ ]:
df_full.head()

Посчитаем количество уникальных вопросв


In [ ]:
uniq_questions = df_full['Id_questions'].nunique()

In [ ]:
uniq_questions

Количество ответов на вопросы

In [ ]:
answer_question = df_full.groupby('Id_questions')['Id_answers'].count()
avg_ans = answer_question.mean()

In [ ]:
answer_question

Среднее количество ответов на вопросы

In [ ]:
print(f'{avg_ans:.2f}')

Посмотрим распределение рейтинга вопросв

In [ ]:
scores_questions = df_full[['Id_questions','Score_questions']].drop_duplicates()['Score_questions']
dstr_questions = scores_questions.value_counts().sort_index()

In [ ]:
print(f'{dstr_questions.to_string()}')

Сделаем то же самое с ответами

In [ ]:
scores_answer = df_full['Score_answers'].dropna()
dstr_answer = scores_answer.value_counts().sort_index()

In [ ]:
print(f'{dstr_answer.to_string()}')

Cгруппируем по тагам и посмотрим, сколько раз встречается каждый тег

In [ ]:
tag_counts = (tags_copy.groupby('Tag')['Id'].nunique().sort_values(ascending=False))
top_tags = tag_counts.head(10)

In [ ]:
print("Топ‑10 тегов по числу вопросов:")
print(top_tags.to_string())

Сделаем отдельные таги для ЯП и остальных, написав список и функцию, добавив их в копию изначального датасета

In [ ]:
lang_tags = {'javascript', 'java', 'c#', 'php', 'python', 'html', 'c++', 'sql', 'kotlin'}
other_tags = {'ios', 'android', 'jquery', 'css', 'asp.net', 'pandas', 'numpy', 'algorithm', 'spark'}

In [ ]:
def category(tag):
    if tag in lang_tags:
        return 'language'
    else:
        return 'other'

In [ ]:
tags_copy['category'] = tags_copy['Tag'].apply(category)

In [ ]:
category_counts = (tags_copy.groupby('category')['Id'].nunique())

Посмотрим какие категории все же встречаются чаще

In [ ]:
print(category_counts.to_string())

Посчитаем количество уникальных вопросов и ответов

In [ ]:
counter_questions = df_full['Id_questions'].nunique()
counter_answers = df_full['Id_answers'].nunique()

In [ ]:
counter_questions

In [ ]:
counter_answers

Посчитаем их длину

In [ ]:
len_questions = df_full['clean_question'].str.len()
len_answers = df_full['clean_answer'].str.len()

Среднее максимальное и минимальное количество символов в вопросах

In [ ]:
print(len_questions.mean())
print(len_questions.max())
print(len_questions.min())

Среднее максимальное и минимальное количество символов в ответах

In [ ]:
print(len_answers.mean())
print(len_answers.max())
print(len_answers.min())

А ВДРУГ ТАМ ЕСТЬ NAN

In [ ]:
print(df_full['clean_question'].isna().sum())
print(df_full['clean_answer'].isna().sum())

In [ ]:
print(df_full['Score_questions'].isna().sum())
print(df_full['Score_answers'].isna().sum())

Вопросики без ответов =(

In [ ]:
q_without_ans = (df_full.groupby('Id_questions')['Id_answers'].count() == 0).sum()
print("Всего вопросов без ответов:", q_without_ans)

Посмотрим топ-25 тагов и построим их график

In [ ]:
top25 = tags_copy['Tag'].value_counts().head(25)

In [ ]:
plt.figure(figsize=(8, 6))
plt.barh(top50.index[::-1], top50.values[::-1])
plt.xlabel("Кол-во Вопросов")
plt.title("Топ-25 Среди Вопросов")
plt.tight_layout()
plt.show()

Хочу чтоб была длина каждого вопроса и ответа в символах

In [ ]:
df_full['len_questions'] = df_full['clean_question'].str.len()
df_full['len_answers'] = df_full['clean_answer'].str.len()

In [ ]:
df_full.head()

Ну и построим графики для них

In [ ]:
plt.figure(figsize=(8, 6))
plt.hist(df_full['len_questions'].clip(upper=1000), bins=50)
plt.xlabel('Кол-во Символов в Вопросе')
plt.ylabel('Кол-во Вопросов')
plt.title('Распределение Длин Вопросов')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
plt.hist(df_full['len_answers'].clip(upper=1000), bins=50)
plt.xlabel('Кол-во Символов в Ответе')
plt.ylabel('Кол-во Ответов')
plt.title('Распределение Длин Ответов')
plt.tight_layout()
plt.show()

# Отчёт

---

## Очистка данных

С помощью BeautifulSoup были удалены теги, с использованием регулярных выражений - Markdown-ссылки

Для снижения шума оставил только англ буквы и пробелы, привёл всё к нижнему регистру

С помощью библиотеки SpaCy провел лемматизацию

И получил два новых столбца с чистыми данными(вопросы и ответы) без лишних символов

---

## Статистические выводы о датасете Stack Overflow

* Кол-во уникальных вопросов: 1 264 216

* Кол-во уникальных ответов: 2 014 516

* Среднее число ответов на вопрос: 1.59

* Длина вопросов в большинстве случаев превышает порог в 1000 символов, что говорит о качестве составления вопросов.

* Длина ответов чаще всего в диапазоне от 100 до 200 символов, что говорит нам о том, что вопросы чаще всего задаются более точечно и узконаправлено.

* Топ-10 тегов вопросов в основном состоит из тегов языков программирования, что говорит о том, что стековерфлоу - сайт для прогеров)

* В датасете находится довольно много NaN значений в столбце ответов, что говорит о том, что не на все вопросы есть ответы.


* Самая популярная тема для вопросов это код на javascript, далее в топе идут также вопросы с тегами языков программирования, а уже ниже находятся категориальные вопросы.

---